<a href="https://colab.research.google.com/github/vard-uhi/Retention-Analysis/blob/master/Human_Resources_Retention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Retention Study - Figuring Out Which Employees May Quit**

#### Loading our HR Database

In [14]:
import numpy as np
import pandas as pd
import pandas.testing as tm
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [15]:
#load the data
from google.colab import files
data_to_load = files.upload()

Saving hr_data.csv to hr_data (1).csv


In [16]:
import io
raw_data = pd.read_csv(io.BytesIO(data_to_load['hr_data.csv']))
raw_data.head()

,employee_id,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,department,salary
0,1003,2,157,3,0,1,0,sales,low
1,1005,5,262,6,0,1,0,sales,medium
2,1486,7,272,4,0,1,0,sales,medium
3,1038,5,223,5,0,1,0,sales,low
4,1057,2,159,3,0,1,0,sales,low


In [17]:
#copying our raw Dataframe, in order to performe manipulations
hr_df = raw_data.copy()

In [18]:
#general overview of the DataFrame
hr_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   employee_id            14999 non-null  int64 
 1   number_project         14999 non-null  int64 
 2   average_montly_hours   14999 non-null  int64 
 3   time_spend_company     14999 non-null  int64 
 4   Work_accident          14999 non-null  int64 
 5   left                   14999 non-null  int64 
 6   promotion_last_5years  14999 non-null  int64 
 7   department             14999 non-null  object
 8   salary                 14999 non-null  object
dtypes: int64(7), object(2)
memory usage: 1.0+ MB


In [29]:
hr_df[hr_df.isnull().any(axis=1)]

,employee_id,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,department,salary



*   We have 2 categorical variables 'department', 'salary'
*   No missing values  
*   Shape of the Dataframe is 14999 rows and 9 columns

#### Loading Evaluation and Employee Satisfaction Data

In [19]:
#load the data
from google.colab import files
data_to_load = files.upload()

Saving employee_satisfaction_evaluation.xlsx to employee_satisfaction_evaluation (1).xlsx


In [20]:
import io
raw_data2 = pd.read_excel(io.BytesIO(data_to_load['employee_satisfaction_evaluation.xlsx']))
raw_data2.head()

,EMPLOYEE #,satisfaction_level,last_evaluation
0,1003,0.38,0.53
1,1005,0.80,0.86
2,1486,0.11,0.88
3,1038,0.72,0.87
4,1057,0.37,0.52


In [21]:
emp_satis_eval = raw_data2.copy()

In [22]:
emp_satis_eval.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   EMPLOYEE #          14999 non-null  int64  
 1   satisfaction_level  14972 non-null  float64
 2   last_evaluation     14972 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 351.7 KB


* We have missing vales.
* No categorical data.
* Shape is 14999 rows and 3 columns

Next we will merge 2 datasets to perform data manipluation, analysis and prediction.

#### Join Tables

In [23]:
main_df = hr_df.set_index('employee_id').join(emp_satis_eval.set_index('EMPLOYEE #'))
main_df = main_df.reset_index()
main_df.head()

,employee_id,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,department,salary,satisfaction_level,last_evaluation
0,1003,2,157,3,0,1,0,sales,low,0.38,0.53
1,1005,5,262,6,0,1,0,sales,medium,0.80,0.86
2,1486,7,272,4,0,1,0,sales,medium,0.11,0.88
3,1038,5,223,5,0,1,0,sales,low,0.72,0.87
4,1057,2,159,3,0,1,0,sales,low,0.37,0.52


In [25]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   employee_id            14999 non-null  int64  
 1   number_project         14999 non-null  int64  
 2   average_montly_hours   14999 non-null  int64  
 3   time_spend_company     14999 non-null  int64  
 4   Work_accident          14999 non-null  int64  
 5   left                   14999 non-null  int64  
 6   promotion_last_5years  14999 non-null  int64  
 7   department             14999 non-null  object 
 8   salary                 14999 non-null  object 
 9   satisfaction_level     14972 non-null  float64
 10  last_evaluation        14972 non-null  float64
dtypes: float64(2), int64(7), object(2)
memory usage: 1.3+ MB


In [26]:
#descriptive statistics of all variables, including categorical
main_df.describe(include='all')

,employee_id,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,department,salary,satisfaction_level,last_evaluation
count,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999,14999,14972.000000,14972.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,3,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sales,low,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4140,7316,NaN,NaN
mean,45424.627575,3.803054,201.050337,3.498233,0.144610,0.238083,0.021268,NaN,NaN,0.612830,0.716125
std,25915.900127,1.232592,49.943099,1.460136,0.351719,0.425924,0.144281,NaN,NaN,0.248714,0.171138
min,1003.000000,2.000000,96.000000,2.000000,0.000000,0.000000,0.000000,NaN,NaN,0.090000,0.360000
25%,22872.500000,3.000000,156.000000,3.000000,0.000000,0.000000,0.000000,NaN,NaN,0.440000,0.560000
50%,45448.000000,4.000000,200.000000,3.000000,0.000000,0.000000,0.000000,NaN,NaN,0.640000,0.720000
75%,67480.500000,5.000000,245.000000,4.000000,0.000000,0.000000,0.000000,NaN,NaN,0.820000,0.870000


####Dealing with missing values

In [33]:
#list of all rows with missing vales
main_df[main_df.isnull().any(axis=1)]

,employee_id,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,department,salary,satisfaction_level,last_evaluation
18,3794,2,160,3,1,1,1,sales,low,NaN,NaN
19,1140,5,262,5,0,1,0,sales,low,NaN,0.89
33,1230,2,140,3,0,1,0,hr,low,NaN,NaN
53,1340,2,132,3,0,1,0,support,low,NaN,NaN
72,22316,2,149,3,0,1,0,product_mng,high,NaN,NaN
92,1581,2,143,3,0,1,0,sales,low,NaN,NaN
107,17376,2,148,3,0,1,0,accounting,medium,NaN,NaN
120,1739,4,158,4,1,1,0,technical,low,NaN,NaN
137,1847,2,129,3,0,1,0,management,low,NaN,NaN
175,32923,4,164,2,0,1,0,sales,low,NaN,NaN


In [32]:
#counting the numer of missing rows
main_df[main_df.isnull().any(axis=1)].count()

employee_id              43
number_project           43
average_montly_hours     43
time_spend_company       43
Work_accident            43
left                     43
promotion_last_5years    43
department               43
salary                   43
satisfaction_level       16
last_evaluation          16
dtype: int64

we see that we have 43 rows with missing vales and they come from 'satisfication level' and 'last evaluation' columns.

To deal with missing vales we will fill in the missing blanks with the aveage values.